In [12]:
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_TABLE_ALIGNMENT
import os
from datetime import datetime

In [13]:
def create_shipping_label(order_data, order_number):
    """
    Create a Word document with shipping label for a single order
    """
    # Create a new document
    doc = Document()
    
    # Set up the document margins (smaller margins for label)
    sections = doc.sections
    for section in sections:
        section.top_margin = Inches(0.5)
        section.bottom_margin = Inches(0.5)
        section.left_margin = Inches(0.5)
        section.right_margin = Inches(0.5)
    
    # Add title
    title = doc.add_heading('Sillage Perfumes', 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Add a line break
    doc.add_paragraph()
    
    # Create a table for the shipping label layout
    table = doc.add_table(rows=2, cols=1)
    table.alignment = WD_TABLE_ALIGNMENT.CENTER
    table.style = None#'Table Grid'
    
    # Set column widths
    table.columns[0].width = Inches(7)
    #table.columns[1].width = Inches(7)
    
    # Get the cells
    receiver_cell = table.cell(0, 0)
    sender_cell = table.cell(1, 0)
    
    
    # SENDER INFORMATION
    sender_paragraph = sender_cell.paragraphs[0]
    sender_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
    
    # Add sender header
    sender_run = sender_paragraph.add_run("FROM:\n")
    sender_run.bold = True
    sender_run.font.size = Pt(18)
    
    # Add sender details
    sender_details = """Sillage Perfumes
Outer Ring Road Doddanekundi
Bengaluru, 560037
Karnataka, India
Phone: 8904620890"""
    
    sender_info_run = sender_paragraph.add_run(sender_details)
    sender_info_run.font.size = Pt(16)
    
    # RECEIVER INFORMATION
    receiver_paragraph = receiver_cell.paragraphs[0]
    receiver_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
    
    # Add receiver header
    receiver_run = receiver_paragraph.add_run("TO:\n")
    receiver_run.bold = True
    receiver_run.font.size = Pt(24)
    
    # Get receiver details from order data
    
    shipping_name = order_data['Shipping Name'].iloc[0] if not pd.isna(order_data['Shipping Name'].iloc[0]) else "N/A"
    shipping_street = order_data['Shipping Street'].iloc[0] if not pd.isna(order_data['Shipping Street'].iloc[0]) else "N/A"
    shipping_city = order_data['Shipping City'].iloc[0] if not pd.isna(order_data['Shipping Street'].iloc[0]) else "N/A"
    shipping_zip = str(order_data['Shipping Zip'].iloc[0] if not pd.isna(order_data['Shipping Street'].iloc[0]) else "N/A").replace(".0","")
    shipping_province = order_data['Shipping Province'].iloc[0] if not pd.isna(order_data['Shipping Street'].iloc[0]) else "N/A"
    shipping_country = order_data['Shipping Country'].iloc[0] if not pd.isna(order_data['Shipping Street'].iloc[0]) else "N/A"
    shipping_phone = str(order_data['Shipping Phone'].iloc[0] if not pd.isna(order_data['Shipping Street'].iloc[0]) else "N/A").replace(".0","")
    
    receiver_details = f"{shipping_name}\n{shipping_street}\n{shipping_city}, {shipping_zip}\n{shipping_province},{shipping_country}\nPhone:{shipping_phone}"
    receiver_info_run = receiver_paragraph.add_run(receiver_details)
    receiver_info_run.font.size = Pt(22)
    
    # Add space between sections
    doc.add_paragraph()
    
    # Order Information Section
    #order_info = doc.add_heading('ORDER DETAILS', level=2)
    #order_info.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Order details table
    order_table = doc.add_table(rows=1, cols=1)
    order_table.style = None #'Table Grid'
    order_table.alignment = WD_TABLE_ALIGNMENT.CENTER
    
    # Order Number
    order_table.cell(0, 0).text = f"Order Number: {str(order_number)}"
    #order_table.cell(0, 1).text = str(order_number)
    
    # Date
    #order_table.cell(1, 0).text = "Date:"
    #order_table.cell(1, 1).text = datetime.now().strftime("%Y-%m-%d")
    
    # Make headers bold
    for i in range(1):
        order_table.cell(i, 0).paragraphs[0].runs[0].bold = True
    
    # Add space
    doc.add_paragraph()
    
    # Items Section
    items_heading = doc.add_heading('ITEMS', level=2)
    items_heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Create items table
    items_table = doc.add_table(rows=1, cols=2)
    items_table.style = 'Table Grid'
    items_table.alignment = WD_TABLE_ALIGNMENT.CENTER
    
    # Add header row
    hdr_cells = items_table.rows[0].cells
    hdr_cells[0].text = 'Item Name'
    hdr_cells[1].text = 'Quantity'
    
    # Make header bold
    for cell in hdr_cells:
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.bold = True
    
    # Group items by name and count quantities
    item_counts = {}
    for _, row in order_data.iterrows():
        item_name = row['Lineitem name']
        if pd.notna(item_name):
            if item_name in item_counts:
                item_counts[item_name] += 1
            else:
                item_counts[item_name] = 1
    
    # Add items to table
    max_items = 7
    item_counter = 0
    for item_name, quantity in item_counts.items():
        item_counter += 1
        if item_counter <= max_items:
            row_cells = items_table.add_row().cells
            row_cells[0].text = str(item_name)
            row_cells[1].text = str(quantity)
        else:
            break
    
    return doc


In [14]:
def process_order_export(file_path):
    """
    Process the order export file and create shipping labels for each order
    """
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if required columns exist
        required_columns = ['Name', 'Shipping Name', 'Shipping Street', 'Lineitem name', 'Shipping City', 'Shipping Zip', 'Shipping Province', 'Shipping Country', 'Shipping Phone']
        missing_columns = [col for col in required_columns if col not in df.columns]
        
        if missing_columns:
            print(f"Error: Missing columns: {missing_columns}")
            print(f"Available columns: {list(df.columns)}")
            return
        
        # Create output directory
        output_dir = "shipping_labels"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Group by order number (Name column)
        grouped_orders = df.groupby('Name')
        
        print(f"Processing {len(grouped_orders)} orders...")
        
        # Process each order
        for order_number, order_data in grouped_orders:
            try:
                # Create shipping label document
                doc = create_shipping_label(order_data, order_number)
                
                # Generate filename (sanitize order number for filename)
                safe_order_number = str(order_number).replace('/', '_').replace('\\', '_')
                filename = f"shipping_label_{safe_order_number}.docx"
                filepath = os.path.join(output_dir, filename)
                
                # Save the document
                doc.save(filepath)
                print(f"Created shipping label: {filename}")
                
            except Exception as e:
                print(f"Error processing order {order_number}: {str(e)}")
        
        print(f"\nAll shipping labels created successfully in '{output_dir}' directory!")
        
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except Exception as e:
        print(f"Error processing file: {str(e)}")

In [15]:
csv_file_path = r"C:\Users\silla\Downloads\orders_export (2).csv"

In [16]:
process_order_export(csv_file_path)

Processing 3 orders...
Created shipping label: shipping_label_#2013.docx
Created shipping label: shipping_label_#2014.docx
Created shipping label: shipping_label_#2015.docx

All shipping labels created successfully in 'shipping_labels' directory!


In [17]:
datetime.now().strftime("%d-%m-%Y")

'27-08-2025'

In [ ]:
if __name__ == "__main__":
    # Specify your input file path
    input_file = "order_export.csv"  # Change this to your actual file path
    
    print("Shipping Label Generator")
    print("=" * 30)
    
    # Check if file exists
    if not os.path.exists(input_file):
        print(f"Please make sure '{input_file}' exists in the current directory.")
        print("Or update the 'input_file' variable with the correct path.")
    else:
        process_order_export(input_file)

In [18]:
import glob
from docx import Document
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

ModuleNotFoundError: No module named 'reportlab'

In [19]:
!pip3 install reportlab

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 13.6 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------------------ --------------- 4.2/7.0 MB 19.7 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 17.4 MB/s  0:00:00

   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ---------